In [1]:
# import library yang dibutuhkan
from sqlalchemy import create_engine
import pymysql
import pymongo
import pandas as pd

In [2]:
# membuat class transformation
class transformation:
    def __init__(self, username="root", password="", host="127.0.0.1", lama_koneksi=3600, mongodb_conn="mongodb://localhost:27017/"):
        sqlEngine = create_engine('mysql+pymysql://' + username + ':' + password + '@' + host, pool_recycle=lama_koneksi)
        connect_mysql = sqlEngine.connect()
        myclient_mongodb = pymongo.MongoClient(mongodb_conn)
        self.mongodb_conn = mongodb_conn
        self.connect_mysql = connect_mysql
        self.myclient_mongodb = myclient_mongodb
        print("Objek class berhasil dibuat")
        
        
    def view_data(self, database_name, tabel_name):
        query_getdata = "select * from" + " " + database_name + "." + tabel_name
        df_databasetabel_read = pd.read_sql_query(query_getdata, self.connect_mysql)
        self.df = df_databasetabel_read
        return df_databasetabel_read
    
    
    def frommysql_tocsv(self, database_name, tabel_name, directory="Documents/dataset_from_database.csv"):
        query_getdata = "select * from" + " " + database_name + "." + tabel_name
        df_databasetabel_write = pd.read_sql_query(query_getdata, self.connect_mysql)
        df_databasetabel_write.to_csv(directory, index = False)
        print('Transformasi ke CSV File berhasil')
    
    
    def frommysql_tomongodb(self, database_name, tabel_name, database_mongo_name, collection_name):
        db = self.myclient_mongodb[database_mongo_name]
        collection = db[collection_name]
        
        query_getdata = "select * from" + " " + database_name + "." + tabel_name
        df_databasetabel_write = pd.read_sql_query(query_getdata, self.connect_mysql)
        df_databasetabel_write = df_databasetabel_write.astype('str')
        
        for row in range(df_databasetabel_write.shape[0]):
            data = df_databasetabel_write.loc[row]
            data_dict = data.to_dict()
            collection.insert_one(data_dict)
            
        print('Transformasi ke No-SQL MongoDB berhasil')

In [3]:
# akses class dan lakukan koneksi ke mysql dan mongodb
username = "root" 
password = "" 
host = "127.0.0.1"
lama_koneksi = 3600
mongodb_conn = "mongodb://localhost:27017/"

tabel = transformation(username, password, host, lama_koneksi, mongodb_conn)

Objek class berhasil dibuat


In [4]:
# melihat tabel terpilih pada database
database_name = "superstore"
tabel_name = "dataset_superstore_simple"

dataframe = tabel.view_data(database_name, tabel_name)
dataframe.head()

,order_id,order_date,customer_id,segment,category,sub_category,sales,quantity,profit
0,CA-2016-152156,2016-11-08,CG-12520,Consumer,Furniture,Bookcases,261.96,2,41.9136
1,CA-2016-152156,2016-11-08,CG-12520,Consumer,Furniture,Chairs,731.94,3,219.582
2,CA-2016-138688,2016-06-12,DV-13045,Corporate,Office Supplies,Labels,14.62,2,6.8714
3,US-2015-108966,2015-10-11,SO-20335,Consumer,Furniture,Tables,957.5775,5,-383.031
4,US-2015-108966,2015-10-11,SO-20335,Consumer,Office Supplies,Storage,22.368,2,2.5164


In [5]:
# melakukan convert dari database mysql ke csv file
database_name = "superstore"
tabel_name = "dataset_superstore_simple"
directory = "Documents/dataset_from_database.csv"

tabel.frommysql_tocsv(database_name, tabel_name, directory)

Transformasi ke CSV File berhasil


In [6]:
# melakukan convert dari database mysql ke mongodb
database_name = "superstore"
tabel_name = "dataset_superstore_simple"
database_mongo_name = "dbproject"
collection_name = "mycollection"

tabel.frommysql_tomongodb(database_name, tabel_name, database_mongo_name, collection_name)

Transformasi ke No-SQL MongoDB berhasil


In [7]:
#melakukan cek apakah database dan collection sudah dibuat
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient[database_mongo_name]
mycol = mydb[collection_name]

#mengambil semua documents pada collection
for data in mycol.find():
    print(data)

{'_id': ObjectId('5fe2ccad3070fa4005369b6a'), 'order_id': 'CA-2016-152156', 'order_date': '2016-11-08', 'customer_id': 'CG-12520', 'segment': 'Consumer', 'category': 'Furniture', 'sub_category': 'Bookcases', 'sales': '261.96', 'quantity': '2', 'profit': '41.9136'}
{'_id': ObjectId('5fe2ccad3070fa4005369b6b'), 'order_id': 'CA-2016-152156', 'order_date': '2016-11-08', 'customer_id': 'CG-12520', 'segment': 'Consumer', 'category': 'Furniture', 'sub_category': 'Chairs', 'sales': '731.94', 'quantity': '3', 'profit': '219.582'}
{'_id': ObjectId('5fe2ccad3070fa4005369b6c'), 'order_id': 'CA-2016-138688', 'order_date': '2016-06-12', 'customer_id': 'DV-13045', 'segment': 'Corporate', 'category': 'Office Supplies', 'sub_category': 'Labels', 'sales': '14.62', 'quantity': '2', 'profit': '6.8714'}
{'_id': ObjectId('5fe2ccad3070fa4005369b6d'), 'order_id': 'US-2015-108966', 'order_date': '2015-10-11', 'customer_id': 'SO-20335', 'segment': 'Consumer', 'category': 'Furniture', 'sub_category': 'Tables', '